# Export Best Models per Section

This notebook trains each canteen section's best-performing model on the
combined train + validation set and serialises the artefacts to disk
(`deployment_models/`) so they can be loaded by the deployment pipeline.

---
## 1 - Imports

In [6]:
!pip install pmdarima

  Using cached pmdarima-2.1.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.5 kB)
Using cached pmdarima-2.1.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (689 kB)


In [7]:
import os
import warnings

import joblib
import numpy as np
import pandas as pd

In [8]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils

In [10]:
from prophet import Prophet
import pmdarima as pm

In [11]:
warnings.filterwarnings('ignore')

In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Change to your own directory
try:
    os.chdir("/content/drive/MyDrive/UAB/FDS/campus-waste-intelligence")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Mounted at /content/drive
Directory changed


---
## 2 - Configuration & Constants

In [13]:
DATA_PATH = 'data/food_waste_cleaned.csv'
MODEL_DIR = 'deployment_models'

TRAIN_END = '2025-07-15'
VAL_END   = '2025-07-25'

LOOKBACK    = 7
LSTM_HIDDEN = 50
LSTM_LAYERS = 2
EPOCHS      = 50
BATCH_SIZE  = 16
LR          = 0.001

os.makedirs(MODEL_DIR, exist_ok=True)

---
## 3 - Load and Aggregate Data

In [14]:
df = pd.read_csv(DATA_PATH, parse_dates=['Date'])

daily_section = (
    df.groupby(['Date', 'Canteen_Section'])['Waste_Weight_kg']
      .sum()
      .reset_index()
      .rename(columns={'Waste_Weight_kg': 'Total_Waste_kg'})
)

daily_section.head()

,Date,Canteen_Section,Total_Waste_kg
0,2025-06-11,A,29.90
1,2025-06-11,B,26.85
2,2025-06-11,C,27.72
3,2025-06-11,D,37.59
4,2025-06-12,A,32.99


In [15]:
daily_wide = (
    daily_section
    .pivot(index='Date', columns='Canteen_Section', values='Total_Waste_kg')
    .fillna(0)
    .sort_index()
    .asfreq('D')
    .fillna(0)
)

sections = daily_wide.columns.tolist()
print(f"Sections found: {sections}")
print(f"Date range: {daily_wide.index.min()} to {daily_wide.index.max()}")

Sections found: ['A', 'B', 'C', 'D']
Date range: 2025-06-11 00:00:00 to 2025-08-10 00:00:00


---
## 4 - Feature Engineering

In [16]:
def create_features_for_series(series: pd.Series) -> pd.DataFrame:
    """Build lag, rolling, and calendar features for a single daily series.

    Parameters
    ----------
    series : pd.Series
        Daily waste series with a DatetimeIndex.

    Returns
    -------
    pd.DataFrame
        Feature matrix with target column ``y``.
    """
    df_ml = pd.DataFrame(index=series.index)
    df_ml['y'] = series.values

    # Calendar features
    df_ml['dayofweek'] = df_ml.index.dayofweek
    df_ml['day']       = df_ml.index.day
    df_ml['month']     = df_ml.index.month
    df_ml['quarter']   = df_ml.index.quarter
    df_ml['weekend']   = (df_ml.index.dayofweek >= 5).astype(int)

    # Lag features
    for lag in [1, 2, 3, 7, 14]:
        df_ml[f'lag_{lag}'] = df_ml['y'].shift(lag)

    # Rolling window features (shifted by 1 to avoid look-ahead)
    shifted = df_ml['y'].shift(1)
    df_ml['rolling_mean_7'] = shifted.rolling(7).mean()
    df_ml['rolling_std_7']  = shifted.rolling(7).std()
    df_ml['rolling_min_7']  = shifted.rolling(7).min()
    df_ml['rolling_max_7']  = shifted.rolling(7).max()
    df_ml['ewm_mean_7']    = shifted.ewm(span=7).mean()

    df_ml.dropna(inplace=True)
    return df_ml

---
## 5 - Build Feature DataFrames & Align Date Range

In [17]:
feature_dfs: dict[str, pd.DataFrame] = {}

for sec in sections:
    feature_dfs[sec] = create_features_for_series(daily_wide[sec])

# Align all sections to the same date window
common_start = max(df_sec.index.min() for df_sec in feature_dfs.values())
common_end   = min(df_sec.index.max() for df_sec in feature_dfs.values())

for sec in sections:
    feature_dfs[sec] = feature_dfs[sec].loc[common_start:common_end]

print(f"Common date range: {common_start.date()} to {common_end.date()}")
print(f"Rows per section:  {len(feature_dfs[sections[0]])}")

Common date range: 2025-06-25 to 2025-08-10
Rows per section:  47


---
## 6 - Train / Validation / Test Masks

In [18]:
ref_index = feature_dfs[sections[0]].index

train_mask = ref_index <= TRAIN_END
val_mask   = (ref_index > TRAIN_END) & (ref_index <= VAL_END)
test_mask  = ref_index > VAL_END

print(f"Train: {train_mask.sum()}  |  Val: {val_mask.sum()}  |  Test: {test_mask.sum()}")

Train: 21  |  Val: 10  |  Test: 16


---
## 7 - Best Model per Section

These are hardcoded from the analysis notebook. If you have the `results_df`
from the section-forecasting notebook you can compute them automatically:

```python
best_models = (
    results_df
    .groupby('Section')
    .apply(lambda g: g.sort_values('RMSE').iloc[0]['Model'])
    .to_dict()
)
```

In [19]:
best_models: dict[str, str] = {
    'A': 'XGBoost',
    'B': 'XGBoost',
    'C': 'Random Forest',
    'D': 'XGBoost',
}

best_models

{'A': 'XGBoost', 'B': 'XGBoost', 'C': 'Random Forest', 'D': 'XGBoost'}

---
## 8 - LSTM Architecture

In [20]:
class LSTMModel(nn.Module):
    """Simple stacked-LSTM regressor."""

    def __init__(
        self,
        input_size: int = 1,
        hidden_size: int = LSTM_HIDDEN,
        num_layers: int = LSTM_LAYERS,
        output_size: int = 1,
    ):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc   = nn.Linear(hidden_size, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out.squeeze()

---
## 9 - Sequence Helper

In [21]:
def create_sequences(
    data: np.ndarray, lookback: int = LOOKBACK
) -> tuple[np.ndarray, np.ndarray]:
    """Slide a window of length *lookback* over *data* to produce (X, y) pairs."""
    xs, ys = [], []
    for i in range(lookback, len(data)):
        xs.append(data[i - lookback : i])
        ys.append(data[i])
    return np.array(xs), np.array(ys)

---
## 10 - Train & Export Function

In [22]:
def train_and_save_section(sec: str, model_name: str) -> None:
    """Train *model_name* on combined train+val data for *sec* and save artefacts."""

    print(f"\nTraining {model_name} for section {sec}")

    df_ml = feature_dfs[sec]
    X = df_ml.drop('y', axis=1)
    y = df_ml['y']

    train_val_idx = train_mask | val_mask
    X_train_val = X[train_val_idx]
    y_train_val = y[train_val_idx]

    feature_columns = X.columns.tolist()

    artifacts: dict = {
        'section': sec,
        'model_name': model_name,
        'feature_columns': feature_columns,
        'lookback': LOOKBACK,
    }

    # ------------------------------------------------------------------
    # Tree-based models
    # ------------------------------------------------------------------
    if model_name in ('XGBoost', 'Random Forest'):
        _train_tree_model(model_name, X_train_val, y_train_val, artifacts, sec)

    # ------------------------------------------------------------------
    # SVM
    # ------------------------------------------------------------------
    elif model_name == 'SVM':
        _train_svm(X_train_val, y_train_val, artifacts, sec)

    # ------------------------------------------------------------------
    # Prophet
    # ------------------------------------------------------------------
    elif model_name == 'Prophet':
        _train_prophet(y_train_val, artifacts, sec)

    # ------------------------------------------------------------------
    # SARIMA
    # ------------------------------------------------------------------
    elif model_name == 'SARIMA':
        _train_sarima(y_train_val, artifacts, sec)

    # ------------------------------------------------------------------
    # LSTM
    # ------------------------------------------------------------------
    elif model_name == 'LSTM':
        _train_lstm(y_train_val, artifacts, sec)

    # ------------------------------------------------------------------
    # Baseline models
    # ------------------------------------------------------------------
    elif model_name in ('Naive', 'Seasonal Naive', 'MA(7)'):
        _save_baseline(y_train_val, artifacts, sec)

    else:
        print(f"  Unsupported model '{model_name}' for section {sec}")

---
## 11 - Per-Model Training Helpers

In [23]:
def _save_artifacts(artifacts: dict, path: str) -> None:
    """Persist artefacts with joblib and print confirmation."""
    joblib.dump(artifacts, path)
    print(f"  Saved -> {path}")

In [24]:
def _train_tree_model(
    model_name: str,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    artifacts: dict,
    sec: str,
) -> None:
    """Train an XGBoost or Random Forest model and save."""
    if model_name == 'XGBoost':
        model = xgb.XGBRegressor(
            n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42,
        )
    else:
        model = RandomForestRegressor(
            n_estimators=100, max_depth=10, random_state=42,
        )

    model.fit(X_train, y_train)
    artifacts['model'] = model
    _save_artifacts(artifacts, f"{MODEL_DIR}/section_{sec}.joblib")

In [25]:
def _train_svm(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    artifacts: dict,
    sec: str,
) -> None:
    """Train an SVM regressor (with scaling) and save."""
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    model = SVR(kernel='rbf', C=100, gamma='scale')
    model.fit(X_scaled, y_train)

    artifacts['model']  = model
    artifacts['scaler'] = scaler
    _save_artifacts(artifacts, f"{MODEL_DIR}/section_{sec}.joblib")

In [26]:
def _train_prophet(
    y_train: pd.Series,
    artifacts: dict,
    sec: str,
) -> None:
    """Train a Prophet model and save."""
    df_prophet = pd.DataFrame({'ds': y_train.index, 'y': y_train.values})

    prophet_model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=False,
    )
    prophet_model.fit(df_prophet)

    artifacts['model'] = prophet_model
    _save_artifacts(artifacts, f"{MODEL_DIR}/section_{sec}.joblib")

In [27]:
def _train_sarima(
    y_train: pd.Series,
    artifacts: dict,
    sec: str,
) -> None:
    """Fit auto-ARIMA with weekly seasonality and save."""
    sarima_model = pm.auto_arima(
        y_train,
        seasonal=True,
        m=7,
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True,
    )

    artifacts['model'] = sarima_model
    _save_artifacts(artifacts, f"{MODEL_DIR}/section_{sec}.joblib")

In [28]:
def _train_lstm(
    y_train: pd.Series,
    artifacts: dict,
    sec: str,
) -> None:
    """Train an LSTM regressor and save weights + metadata."""

    X_seq, y_seq = create_sequences(y_train.values, lookback=LOOKBACK)
    X_tensor = torch.tensor(X_seq, dtype=torch.float32).unsqueeze(-1)
    y_tensor = torch.tensor(y_seq, dtype=torch.float32)

    lstm_model = LSTMModel()
    criterion  = nn.MSELoss()
    optimizer  = optim.Adam(lstm_model.parameters(), lr=LR)

    dataset = data_utils.TensorDataset(X_tensor, y_tensor)
    loader  = data_utils.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    for epoch in range(EPOCHS):
        lstm_model.train()
        for xb, yb in loader:
            optimizer.zero_grad()
            loss = criterion(lstm_model(xb), yb)
            loss.backward()
            optimizer.step()

    # Save PyTorch state dict
    weights_path = f"{MODEL_DIR}/section_{sec}_lstm.pth"
    torch.save(lstm_model.state_dict(), weights_path)
    print(f"  Saved weights -> {weights_path}")

    # Save metadata (model loaded separately from .pth)
    artifacts['model'] = None
    artifacts['state_dict_path'] = f"section_{sec}_lstm.pth"
    _save_artifacts(artifacts, f"{MODEL_DIR}/section_{sec}_lstm_meta.joblib")

In [29]:
def _save_baseline(
    y_train: pd.Series,
    artifacts: dict,
    sec: str,
) -> None:
    """Store the last 14 actuals so baseline models can forecast at deploy time."""
    artifacts['last_values'] = y_train[-14:].tolist()
    artifacts['last_dates']  = y_train.index[-14:].strftime('%Y-%m-%d').tolist()
    _save_artifacts(artifacts, f"{MODEL_DIR}/section_{sec}_baseline.joblib")

---
## 12 - Run Export Loop

In [30]:
for sec, model_name in best_models.items():
    train_and_save_section(sec, model_name)

print("\nAll models exported successfully.")


Training XGBoost for section A
  Saved -> deployment_models/section_A.joblib

Training XGBoost for section B
  Saved -> deployment_models/section_B.joblib

Training Random Forest for section C
  Saved -> deployment_models/section_C.joblib

Training XGBoost for section D
  Saved -> deployment_models/section_D.joblib

All models exported successfully.


---
## 13 - Verify Saved Artefacts

In [31]:
saved_files = sorted(os.listdir(MODEL_DIR))
print(f"Files in '{MODEL_DIR}/' ({len(saved_files)}):\n")
for f in saved_files:
    size_kb = os.path.getsize(os.path.join(MODEL_DIR, f)) / 1024
    print(f"  {f:40s}  {size_kb:>8.1f} KB")

Files in 'deployment_models/' (4):

  section_A.joblib                             165.3 KB
  section_B.joblib                             157.5 KB
  section_C.joblib                             305.5 KB
  section_D.joblib                             155.7 KB
